# Subqueries

> Udviklet af Thomas Lange & Mick Ahlmann Brun

Mere info: [https://github.com/M1ckB/T-SQL](https://github.com/M1ckB/T-SQL)

Version 1.0 2023-01-25

Laboratoriet kræver:

- En understøttet version af SQL Server
- En Stack Overflow database: [Brent Ozar](https://www.BrentOzar.com/go/querystack) (medium)

Læs mere om subqueries i Microsofts T-SQL reference:

- [https://learn.microsoft.com/en-us/sql/relational-databases/performance/subqueries?view=sql-server-ver16](https://learn.microsoft.com/en-us/sql/relational-databases/performance/subqueries?view=sql-server-ver16)


## Indholdsfortegnelse

- [Introduktion til subqueries](#Introduktion-til-subqueries)
- [Uafhængige subqueries](#Uafhængige-subqueries)
  - [Skalar subquery](#Skalar-subquery)
  - [Flerværdi subquery](#Flerværdi-subquery)
- [Korrelerede subqueries](#Korrelerede-subqueries)
  - [EXISTS-prædikatet](#EXISTS-prædikatet)
- [Hovedpointer](#Hovedpointer)


## Introduktion til subqueries


In [ ]:
/* Eksempel */

## Uafhængige subqueries

Beskrivelse...


### Skalar subquery

Beskrivelse...


In [ ]:
/* Eksempel */


### *Tid til opgaver...*

Lav opgave X i [opgavehæftet](Subqueries.sql).


### Flerværdi subquery

Beskrivelse...


In [ ]:
/* Eksempel */

### *Tid til opgaver...*

Lav opgave X i [opgavehæftet](Subqueries.sql).


## Korrelerede subqueries

Beskrivelse...


In [ ]:
/* Eksempel */

### *Tid til opgaver...*

Lav opgave X i [opgavehæftet](Subqueries.sql).


### `EXISTS`-prædikatet

Beskrivelse...


In [ ]:
/* Eksempel */

### *Tid til opgaver...*

Lav opgave X i [opgavehæftet](Subqueries.sql).


## Hovedpointer

- HOVEDPOINTE


## Licens

Creative Commons Attribution-ShareAlike 4.0 International (CC BY-SA 4.0)

Mere info: [https://creativecommons.org/licenses/by-sa/4.0/](https://creativecommons.org/licenses/by-sa/4.0/)

Du kan frit:

- Dele: kopiere og distribuere materialet via ethvert medium og i ethvert format
- Tilpasse: remixe, redigere og bygge på materialet til ethvert formål, selv erhvervsmæssigt

Under følgende betingelser:

- Kreditering: Du skal kreditere, dele et link til licensen og indikere om der er lavet ændringer.
- Del på samme vilkår: Hvis du remixer, redigerer eller bygger på materialet, så skal dine bidrag
  distribueres under samme licens som den originale.
